# Introduction

The *pyveg* package contains some useful functions for interacting with the Python API of Google Earth Engine.

However, before we can use GEE, we need to authenticate (assuming we have an account).

In [ ]:
import ee
ee.Authenticate()

## Pipelines, Sequences, and Modules

In pyveg, we have the concept of a "Pipeline" for downloading and processing data from GEE.

A Pipeline is composed of one or more Sequences, which are in turn composed of Modules.

A Module is an class designed for one specific task (e.g. "download vegetation data from GEE", or "calculate network centrality of binary images"), and they are generally grouped into Sequences such that one Module will work on the output of the previous one.  
So our standard Pipeline has:
* A vegetation Sequence consisting of VegetationDownloader, VegetationImageProcessor, NetworkCentralityCalculator, and NDVICalculator.   
* A weather Sequence consisting of WeatherDownloader, WeatherImageToJSON
* A combiner Sequence consisting of a single combiner Module, that takes the outputs of the other two Sequences and produces a final output file.

### Running the full pipeline from the command-line

For the second part of this notebook will will demonstrate running individual Modules and Sequences, but the majority of users will probably just want to run the full Pipeline for their selected location/collection/date range, so we will cover that first.

We have a couple of "entrypoints" (i.e. command-line commands) linked to functions in some pyveg scripts to help do this.  
* To configure and run a downloading-and-processing pipeline we run the command `pyveg_run_pipeline --config_file <some-config-file>`
* To generate the config file in the above command we have the command `pyveg_generate_config`.

Both these can accept multiple command-line arguments, and these can be seen with the `--help` argument:

In [5]:
!pyveg_generate_config --help

usage: pyveg_generate_config [-h] [--configs_dir CONFIGS_DIR]
                             [--collection_name COLLECTION_NAME]
                             [--output_dir OUTPUT_DIR] [--test_mode]
                             [--latitude LATITUDE] [--longitude LONGITUDE]
                             [--country COUNTRY] [--start_date START_DATE]
                             [--end_date END_DATE]
                             [--time_per_point TIME_PER_POINT]
                             [--run_mode RUN_MODE] [--n_threads N_THREADS]

create a config file for running pyveg_pipeline

optional arguments:
  -h, --help            show this help message and exit
  --configs_dir CONFIGS_DIR
                        path to directory containing config files
  --collection_name COLLECTION_NAME
                        collection name (e.g. 'Sentinel2')
  --output_dir OUTPUT_DIR
                        Directory for local output data
  --test_mode           Run in test mode, over fewer months and with

For `pyveg_generate_config` any parameters it needs that are not provided as command-line arguments will be requested from the user, and the various allowed options will be provided, along with (in most cases) default values that will be used if the user just presses "enter".
However, although just running `pyveg_generate_config` with no arguments and then responding to the prompts is probably the easiest way to run it on the command line, this doesn't seem to work so well with Jupyter, so let's just provide all the arguments it needs:

In [1]:
!pyveg_generate_config --configs_dir ../../pyveg/configs --collection_name Sentinel2 --output_dir ./ --test_mode --latitude 11.58 --longitude 27.94 --country Sudan --start_date 2019-01-01 --end_date 2019-04-01 --time_per_point 1m --run_mode local --n_threads 2


    output_location ./Sentinel2-11.58N-27.94E-Sudan
    collection: Sentinel2
    latitude: 11.58
    longitude: 27.94
    country: Sudan
    start_date: 2019-01-01
    end_date: 2019-04-01
    time_per_point: 1m
    run_mode: local
    n_threads: 2
    
Wrote file 
  ../../pyveg/configs/testconfig_Sentinel2_11.58N_27.94E_Sudan_2019-01-01_2019-04-01_1m_local.py
We recommend that you add and commit this to your version control repository.

To run pyveg using this configuration, do:

pyveg_run_pipeline --config_file ../../pyveg/configs/testconfig_Sentinel2_11.58N_27.94E_Sudan_2019-01-01_2019-04-01_1m_local.py




We can see from the output that a new config file has been written, and the command we should use to run with it.

In [2]:
!pyveg_run_pipeline --config_file ../../pyveg/configs/testconfig_Sentinel2_11.58N_27.94E_Sudan_2019-01-01_2019-04-01_1m_local.py



2020-09-04 15:43:20,776 [INFO] Sentinel2: setting collection_name to COPERNICUS/S2
2020-09-04 15:43:20,776 [INFO] Sentinel2: setting data_type to vegetation
2020-09-04 15:43:20,776 [INFO] Sentinel2: setting RGB_bands to ['B4', 'B3', 'B2']
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting NIR_band to B8
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting mask_cloud to True
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting cloudy_pix_frac to 50
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting cloudy_pix_flag to CLOUDY_PIXEL_PERCENTAGE
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting min_date to 2015-01-01
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting max_date to 2020-09-04
2020-09-04 15:43:20,777 [INFO] Sentinel2: setting time_per_point to 1m
2020-09-04 15:43:20,838 [INFO] Sentinel2: setting time_per_point to 1m
2020-09-04 15:43:20,839 [INFO] VegetationImageProcessor: setting run_mode to local
2020-09-04 15:43:20,839 [INFO] NetworkCentralityCalculator: setting n_threads to 2
2020-09-

Saved image './Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_Sentinel2/2019-03-16/PROCESSED/2019-03-16_27.94_11.58_NDVI.png'
Saved image './Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_Sentinel2/2019-03-16/PROCESSED/2019-03-16_27.94_11.58_BWNDVI.png'
2020-09-04 15:47:32,484 [INFO] Sentinel2_NetworkCentralityCalculator: Running local
INFO:pyveg_logger:Sentinel2_NetworkCentralityCalculator: Running local
2020-09-04 15:47:32,488 [INFO] Sentinel2_NetworkCentralityCalculator: processing 2019-01-16
INFO:pyveg_logger:Sentinel2_NetworkCentralityCalculator: processing 2019-01-16
2020-09-04 15:47:32,675 [INFO] Sentinel2_NetworkCentralityCalculator found 289 sub-images
INFO:pyveg_logger:Sentinel2_NetworkCentralityCalculator found 289 sub-images
2020-09-04 15:48:03,972 [INFO] 
 Consolidating json from all subimages
INFO:pyveg_logger:
 Consolidating json from all subimages
2020-09-04 15:48:04,006 [INFO] Sentinel2_NetworkCentralityCalculator: processing 201

2020-09-04 15:49:56,457 [INFO] ERA5_WeatherDownloader: Will download to ./Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_ERA5/1986-11-16/RAW
INFO:pyveg_logger:ERA5_WeatherDownloader: Will download to ./Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_ERA5/1986-11-16/RAW
2020-09-04 15:49:56,462 [INFO] ERA5_WeatherDownloader: download succeeded for date range ['1986-11-01', '1986-12-01']
INFO:pyveg_logger:ERA5_WeatherDownloader: download succeeded for date range ['1986-11-01', '1986-12-01']
2020-09-04 15:50:00,675 [INFO] ERA5_WeatherDownloader: Will download to ./Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_ERA5/1986-12-16/RAW
INFO:pyveg_logger:ERA5_WeatherDownloader: Will download to ./Sentinel2-11.58N-27.94E-Sudan__2020-09-04_15-43-20/gee_27.94_11.58_ERA5/1986-12-16/RAW
2020-09-04 15:50:00,679 [INFO] ERA5_WeatherDownloader: download succeeded for date range ['1986-12-01', '1987-01-01']
INFO:pyveg_logger:ERA5_WeatherDownloader:


So we just:
* Downloaded some Sentinel2 images from GEE
* Converted these raw tif images into RGB png, greyscale NDVI png, and black-and-white binarized png images.
* Split the above pngs into 50x50 sub-images
* Calculated the Network Centrality and total NDVI of each sub-image
* Downloaded some ERA5 weather data from GEE
* Read off the values of precipitation and temperature from these tifs
* Combined the vegetation and weather data into one output file

The final outpuf file is called "results_summary.json" and contains some metadata describing the configuration, and time-series data for the vegetation and weather.

# (Optional) Running the pieces individually

Though the above method is the easiest way to get up-and-running, some users may be interested in running the components of pyveg individually.

In [3]:
from pyveg.src.download_modules import VegetationDownloader

In [25]:
# instantiate this Module:
vd = VegetationDownloader("Sentinel2_download")

A lot of the parameters we need to configure this Module are in the `configs/collections.py` file - there is a large dictionary containing values for e.g. Sentinel 2.

In [26]:
from pyveg.configs.collections import data_collections
s2_config = data_collections["Sentinel2"]
print(s2_config)

{'collection_name': 'COPERNICUS/S2', 'data_type': 'vegetation', 'RGB_bands': ['B4', 'B3', 'B2'], 'NIR_band': 'B8', 'mask_cloud': True, 'cloudy_pix_frac': 50, 'cloudy_pix_flag': 'CLOUDY_PIXEL_PERCENTAGE', 'min_date': '2015-01-01', 'max_date': '2020-09-04', 'time_per_point': '1m'}


we also need to specify the coordinates we want to look at (in ***(long,lat)*** format) - let's look at one of our locations in the Sahel:

In [27]:
coords = [28.37,11.12]

And we need to choose a date range.  If we are looking at vegetation data as in this case, we will take the median of all images available within this date range (after filtering out cloudy ones).

In [28]:
date_range = ["2018-06-01","2018-07-01"]

We also need to set an output location to store the files.  We can just use a temporary directory.   The downloaded files will go into a subdirectory of this called "RAW", and then into further subdirectories per mid-point of each date sub-range we're looking at.   Here, we are just looking at one month, and the midpoint will be "2018-06-16".

In [29]:
if os.name == "posix":
    TMPDIR = "/tmp"
else:
    TMPDIR = "%TMP%"
    
output_location = os.path.join(TMPDIR,"gee_download_example")
output_location_type = "local" # other alternative currently possible is `azure` for MS Azure cloud, if setup

Now we're ready to configure the module:

In [30]:

# we could go through all the key,value pairs in the s2_config dict setting them all
# individually, but lets do them all at once
vd.set_parameters(s2_config)
vd.coords = coords
vd.date_range = date_range
vd.output_location = output_location
vd.output_location_type = output_location_type
vd.configure()
print(vd)

2020-09-04 16:48:54,688 [INFO] Sentinel2_download: setting collection_name to COPERNICUS/S2
INFO:pyveg_logger:Sentinel2_download: setting collection_name to COPERNICUS/S2
2020-09-04 16:48:54,690 [INFO] Sentinel2_download: setting data_type to vegetation
INFO:pyveg_logger:Sentinel2_download: setting data_type to vegetation
2020-09-04 16:48:54,693 [INFO] Sentinel2_download: setting RGB_bands to ['B4', 'B3', 'B2']
INFO:pyveg_logger:Sentinel2_download: setting RGB_bands to ['B4', 'B3', 'B2']
2020-09-04 16:48:54,695 [INFO] Sentinel2_download: setting NIR_band to B8
INFO:pyveg_logger:Sentinel2_download: setting NIR_band to B8
2020-09-04 16:48:54,698 [INFO] Sentinel2_download: setting mask_cloud to True
INFO:pyveg_logger:Sentinel2_download: setting mask_cloud to True
2020-09-04 16:48:54,700 [INFO] Sentinel2_download: setting cloudy_pix_frac to 50
INFO:pyveg_logger:Sentinel2_download: setting cloudy_pix_frac to 50
2020-09-04 16:48:54,701 [INFO] Sentinel2_download: setting cloudy_pix_flag to CL

        [Module]: Sentinel2_download 
        depends_on: []
        is_configured: True
        is_finished: False
        run_status: {'succeeded': 0, 'failed': 0, 'incomplete': 0}
        collection_name: COPERNICUS/S2
        data_type: vegetation
        RGB_bands: ['B4', 'B3', 'B2']
        NIR_band: B8
        mask_cloud: True
        cloudy_pix_frac: 50
        cloudy_pix_flag: CLOUDY_PIXEL_PERCENTAGE
        min_date: 2015-01-01
        max_date: 2020-09-04
        time_per_point: 1m
        coords: [28.37, 11.12]
        date_range: ['2018-06-01', '2018-07-01']
        output_location: /tmp
        output_location_type: local
        region_size: 0.08
        scale: 10
        replace_existing_files: False
        num_files_per_point: 4




The Module is all configured and ready-to-go!

In [15]:
vd.run()

2020-09-04 16:27:16,614 [INFO] Sentinel2: Will download to /tmp/2018-06-16/RAW
INFO:pyveg_logger:Sentinel2: Will download to /tmp/2018-06-16/RAW
2020-09-04 16:27:16,659 [INFO] Sentinel2: download succeeded for date range ['2018-06-01', '2018-07-01']
INFO:pyveg_logger:Sentinel2: download succeeded for date range ['2018-06-01', '2018-07-01']


{'succeeded': 1, 'failed': 0, 'incomplete': 0}

There should now be some files in the output location:

In [20]:
os.listdir(os.path.join(TMPDIR,"2018-06-16","RAW"))

['download.NDVI.tif', 'download.B4.tif', 'download.B3.tif', 'download.B2.tif']

So we have one .tif file per band.   

The next Module we would normally run in the vegetation Sequence is the VegetationImageProcessor that will take these tif files and produce png images from them.  This includes histogram equalization, adaptive thresholding and median filtering on an input image, to give us binary NDVI images.  It then divides these into 50x50 sub-images.

In [33]:
from pyveg.src.processor_modules import VegetationImageProcessor
vip = VegetationImageProcessor("Sentinel2_img_processor")
vip.set_parameters(s2_config)
vip.coords = coords


2020-09-04 16:50:40,461 [INFO] Sentinel2_img_processor: setting collection_name to COPERNICUS/S2
INFO:pyveg_logger:Sentinel2_img_processor: setting collection_name to COPERNICUS/S2
2020-09-04 16:50:40,463 [INFO] Sentinel2_img_processor: setting data_type to vegetation
INFO:pyveg_logger:Sentinel2_img_processor: setting data_type to vegetation
2020-09-04 16:50:40,464 [INFO] Sentinel2_img_processor: setting RGB_bands to ['B4', 'B3', 'B2']
INFO:pyveg_logger:Sentinel2_img_processor: setting RGB_bands to ['B4', 'B3', 'B2']
2020-09-04 16:50:40,467 [INFO] Sentinel2_img_processor: setting NIR_band to B8
INFO:pyveg_logger:Sentinel2_img_processor: setting NIR_band to B8
2020-09-04 16:50:40,469 [INFO] Sentinel2_img_processor: setting mask_cloud to True
INFO:pyveg_logger:Sentinel2_img_processor: setting mask_cloud to True
2020-09-04 16:50:40,470 [INFO] Sentinel2_img_processor: setting cloudy_pix_frac to 50
INFO:pyveg_logger:Sentinel2_img_processor: setting cloudy_pix_frac to 50
2020-09-04 16:50:40,

The only other things we need to set are the `input_location` (which will be the `output_location` from the downloader), and the `output_location` (which we will put as the same as the downloader's one - the results of this will go into different subdirectories of the date-named subdirectories).

In [34]:
vip.input_location = vd.output_location
vip.output_location = vd.output_location
vip.configure()
print(vip)

        [Module]: Sentinel2_img_processor 
        depends_on: []
        is_configured: True
        is_finished: False
        run_status: {'succeeded': 0, 'failed': 0, 'incomplete': 0}
        collection_name: COPERNICUS/S2
        data_type: vegetation
        RGB_bands: ['B4', 'B3', 'B2']
        NIR_band: B8
        mask_cloud: True
        cloudy_pix_frac: 50
        cloudy_pix_flag: CLOUDY_PIXEL_PERCENTAGE
        min_date: 2015-01-01
        max_date: 2020-09-04
        time_per_point: 1m
        coords: [28.37, 11.12]
        input_location: /tmp
        output_location: /tmp
        replace_existing_files: False
        num_files_per_point: 3
        input_location_type: local
        output_location_type: local
        dates_to_process: []
        run_mode: local
        n_batch_tasks: -1
        batch_task_dict: {}
        timeout: 30
        region_size: 0.08
        split_RGB_images: True
        input_location_subdirs: ['RAW']
        output_location_subdirs: ['PROCESSE

In [35]:
vip.run()

2020-09-04 16:50:48,562 [INFO] Sentinel2_img_processor: Running local
INFO:pyveg_logger:Sentinel2_img_processor: Running local
2020-09-04 16:50:48,565 [INFO] Sentinel2_img_processor processing files in /tmp/2018-06-16/RAW
INFO:pyveg_logger:Sentinel2_img_processor processing files in /tmp/2018-06-16/RAW
2020-09-04 16:50:48,567 [INFO] ['download.NDVI.tif', 'download.B4.tif', 'download.B3.tif', 'download.B2.tif']
INFO:pyveg_logger:['download.NDVI.tif', 'download.B4.tif', 'download.B3.tif', 'download.B2.tif']
2020-09-04 16:50:48,569 [INFO] Sentinel2_img_processor: Saving RGB image for 2018-06-16 28.37_11.12
INFO:pyveg_logger:Sentinel2_img_processor: Saving RGB image for 2018-06-16 28.37_11.12
2020-09-04 16:50:55,482 [INFO] Will save image to /tmp/2018-06-16/PROCESSED / 2018-06-16_28.37_11.12_RGB.png
INFO:pyveg_logger:Will save image to /tmp/2018-06-16/PROCESSED / 2018-06-16_28.37_11.12_RGB.png


Saved image '/tmp/2018-06-16/PROCESSED/2018-06-16_28.37_11.12_RGB.png'
Saved image '/tmp/2018-06-16/PROCESSED/2018-06-16_28.37_11.12_NDVI.png'
Saved image '/tmp/2018-06-16/PROCESSED/2018-06-16_28.37_11.12_BWNDVI.png'


2020-09-04 16:51:10,652 [INFO] Sentinel2_img_processor: Sophos_Support_4eyy5C not a date string
INFO:pyveg_logger:Sentinel2_img_processor: Sophos_Support_4eyy5C not a date string
2020-09-04 16:51:10,655 [INFO] Sentinel2_img_processor: Sophos_Support_5QgQRN not a date string
INFO:pyveg_logger:Sentinel2_img_processor: Sophos_Support_5QgQRN not a date string
2020-09-04 16:51:10,657 [INFO] Sentinel2_img_processor: Sophos_Support_OOCZM4 not a date string
INFO:pyveg_logger:Sentinel2_img_processor: Sophos_Support_OOCZM4 not a date string
2020-09-04 16:51:10,659 [INFO] Sentinel2_img_processor: Sophos_Support_TW4wl2 not a date string
INFO:pyveg_logger:Sentinel2_img_processor: Sophos_Support_TW4wl2 not a date string
2020-09-04 16:51:10,661 [INFO] Sentinel2_img_processor: Sophos_Support_oqJmN5 not a date string
INFO:pyveg_logger:Sentinel2_img_processor: Sophos_Support_oqJmN5 not a date string
2020-09-04 16:51:10,662 [INFO] Sentinel2_img_processor: Sophos_Support_w6ATPr not a date string
INFO:pyve

{'succeeded': 1, 'failed': 0, 'incomplete': 0}

This should have created two new subdirectories: "PROCESSED" contains the full-size RGB, greyscale, and black-and-white images (the first of these using the RGB bands, and the latter two based on the NDVI band).  "SPLIT" contains the 50x50 sub-images.

In [38]:
os.listdir(os.path.join(output_location,"2018-06-16","PROCESSED"))

['2018-06-16_28.37_11.12_NDVI.png',
 '2018-06-16_28.37_11.12_BWNDVI.png',
 '2018-06-16_28.37_11.12_RGB.png']

## Calculating network centrality

The next step in the standard vegetation sequence is the calculation of "offset50", which is related to the "connectedness" of the vegetation in the black-and-white NDVI sub-images.

In [39]:
from pyveg.src.processor_modules import NetworkCentralityCalculator
ncc = NetworkCentralityCalculator("Sentinel2_ncc")
ncc.set_parameters(s2_config)
ncc.input_location = vip.output_location
ncc.output_location = vip.output_location # same output location again - will create a 'JSON' subdir
ncc.configure()
print(ncc)

2020-09-04 17:05:26,290 [INFO] Sentinel2_ncc: setting collection_name to COPERNICUS/S2
INFO:pyveg_logger:Sentinel2_ncc: setting collection_name to COPERNICUS/S2
2020-09-04 17:05:26,293 [INFO] Sentinel2_ncc: setting data_type to vegetation
INFO:pyveg_logger:Sentinel2_ncc: setting data_type to vegetation
2020-09-04 17:05:26,294 [INFO] Sentinel2_ncc: setting RGB_bands to ['B4', 'B3', 'B2']
INFO:pyveg_logger:Sentinel2_ncc: setting RGB_bands to ['B4', 'B3', 'B2']
2020-09-04 17:05:26,296 [INFO] Sentinel2_ncc: setting NIR_band to B8
INFO:pyveg_logger:Sentinel2_ncc: setting NIR_band to B8
2020-09-04 17:05:26,298 [INFO] Sentinel2_ncc: setting mask_cloud to True
INFO:pyveg_logger:Sentinel2_ncc: setting mask_cloud to True
2020-09-04 17:05:26,300 [INFO] Sentinel2_ncc: setting cloudy_pix_frac to 50
INFO:pyveg_logger:Sentinel2_ncc: setting cloudy_pix_frac to 50
2020-09-04 17:05:26,303 [INFO] Sentinel2_ncc: setting cloudy_pix_flag to CLOUDY_PIXEL_PERCENTAGE
INFO:pyveg_logger:Sentinel2_ncc: setting cl

        [Module]: Sentinel2_ncc 
        depends_on: []
        is_configured: True
        is_finished: False
        run_status: {'succeeded': 0, 'failed': 0, 'incomplete': 0}
        collection_name: COPERNICUS/S2
        data_type: vegetation
        RGB_bands: ['B4', 'B3', 'B2']
        NIR_band: B8
        mask_cloud: True
        cloudy_pix_frac: 50
        cloudy_pix_flag: CLOUDY_PIXEL_PERCENTAGE
        min_date: 2015-01-01
        max_date: 2020-09-04
        time_per_point: 1m
        input_location: /tmp
        output_location: /tmp
        replace_existing_files: False
        num_files_per_point: 1
        input_location_type: local
        output_location_type: local
        dates_to_process: []
        run_mode: local
        n_batch_tasks: -1
        batch_task_dict: {}
        timeout: 30
        n_threads: 4
        n_sub_images: -1
        input_location_subdirs: ['SPLIT']
        output_location_subdirs: ['JSON', 'NC']




One other setting that we might want to change is the number of sub-images per full-size-image for which we do the network centrality calculation.   There are 289 sub-images per full-size-image, and it can be quite time-consuming to process all of them (even though some parallization is implemented - see `n_threads` argument).   We can set this to a smaller number for testing purposes.

In [41]:
ncc.n_sub_images = 10

In [42]:
ncc.run()

2020-09-04 17:11:29,863 [INFO] Sentinel2_ncc: Running local
INFO:pyveg_logger:Sentinel2_ncc: Running local
2020-09-04 17:11:29,874 [INFO] Sentinel2_ncc: processing 2018-06-16
INFO:pyveg_logger:Sentinel2_ncc: processing 2018-06-16
2020-09-04 17:11:30,080 [INFO] Sentinel2_ncc found 289 sub-images
INFO:pyveg_logger:Sentinel2_ncc found 289 sub-images
2020-09-04 17:11:45,461 [INFO] 
 Consolidating json from all subimages
INFO:pyveg_logger:
 Consolidating json from all subimages
2020-09-04 17:11:45,496 [INFO] Sentinel2_ncc: Sophos_Support_4eyy5C not a date string
INFO:pyveg_logger:Sentinel2_ncc: Sophos_Support_4eyy5C not a date string
2020-09-04 17:11:45,498 [INFO] Sentinel2_ncc: Sophos_Support_5QgQRN not a date string
INFO:pyveg_logger:Sentinel2_ncc: Sophos_Support_5QgQRN not a date string
2020-09-04 17:11:45,502 [INFO] Sentinel2_ncc: Sophos_Support_OOCZM4 not a date string
INFO:pyveg_logger:Sentinel2_ncc: Sophos_Support_OOCZM4 not a date string
2020-09-04 17:11:45,504 [INFO] Sentinel2_ncc:

{'succeeded': 1, 'failed': 0, 'incomplete': 0}

We should now have a json file in the output directory:

In [45]:
os.listdir(os.path.join(output_location,"2018-06-16","JSON","NC"))

['network_centralities.json']

In [48]:
import json
j=json.load(open(os.path.join(output_location,"2018-06-16","JSON","NC","network_centralities.json")))


list

The contents of the json file is a list (one entry per sub-image) of dictionaries, and the dictionary keys includ latitude, longitude of the sub-image, as well as "offset50".

In [50]:
j[0].keys()

dict_keys(['slope', 'offset', 'offset50', 'mean', 'std', 'feature_vec', 'date', 'latitude', 'longitude'])

In [51]:
j[0]["offset50"]

-794.0